# Take in raw fluorescence images and compute a scalar fluorescence score

In [7]:
%load_ext autoreload
%autoreload 2

import zarr
import dask.array as da
import pandas as pd
import numpy as np
from scipy import stats
from scipy import ndimage as ndi

from pathlib import Path
from bsccm import BSCCM 


home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'

dataset = BSCCM(data_root)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
fluor_dataframe = pd.DataFrame()
# copy over global insex
fluor_dataframe.insert(0, 'global_index', dataset.index_dataframe.global_index.to_list())

for index in dataset.index_dataframe.global_index:
    index = int(index)
    print('{}          \r'.format(index), end='')

    for ch_index, fluor_channel in enumerate(dataset.global_metadata['fluorescence']['channel_names']):    
        #compute fluorescence
        image = dataset.read_image(index, contrast_type='fluor', channel=fluor_channel).astype(np.float32)

        yy, xx = np.meshgrid(np.arange(image.shape[0]), np.arange(image.shape[0]))
        fluor_mask = np.sqrt((yy - image.shape[0]/2) ** 2 + (xx - image.shape[1]/2) ** 2) < image.shape[0]/2
        fluor_mask_pixels = image[fluor_mask]
        fluor_background_pixels = image[np.logical_not(fluor_mask)]
        
        total_fluor = np.mean(fluor_mask_pixels)
        fluor_dataframe.loc[index, fluor_channel + '_total_raw'] = total_fluor
        total_background = np.mean(fluor_background_pixels)
        fluor_dataframe.loc[index, fluor_channel + '_background'] = total_background


In [9]:
fluor_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)